In [1]:
import vcf

import os
import gzip

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use("ggplot")

In [2]:
# VCF data loading "hello world"
system_path = r"C:\Users\uniqu\Adaptation\github repos" \
              + "\Bioinformatics-Neural Networks for Genomic Risk"
system_path = system_path + r"\DawleyRats"
vcf_file_name_options = [r"allChr.allSamps.90DR2.maf01.hweE7.noIBD.CharlesRiverOnly.vcf.gz",
                         r"allChr.allSamps.90DR2.maf01.hweE7.noIBD.HarlanOnly.vcf.gz"]
vcf_file_name = vcf_file_name_options[0] # "...vcf.gz"

vcf_file_path = os.path.join(system_path, vcf_file_name)

vcf_reader = vcf.Reader(filename=vcf_file_path, compressed=True)

try:
    print("VCF data loaded successfully...\n")
    print(f"Metadata:\n{vcf_reader.metadata}")
except: 
    print("Failed to load VCF data.")

VCF data loaded successfully...

Metadata:
OrderedDict([('fileformat', 'VCFv4.2'), ('fileDate', '20180201'), ('source', ['PLINKv1.90'])])


--- 

## Understanding the PyVCF library

Resources Used:
- [[PyVCF docs]](https://pyvcf.readthedocs.io/en/latest/)
- [PyVCF Tutorial: Michal Linial (Jan, 2020). *Quantitative Biological Research with Python*.](https://youtu.be/jWu_nxlS5Vc) (ends @ 12 minutes)

###  `vcf.Reader` [[docs]](https://pyvcf.readthedocs.io/en/latest/API.html#vcf-reader)

In [3]:
# Read the first n characters of the .gz vcf file.
with gzip.open(vcf_file_path, 'rt') as f:
    print(f.read(n := int(1e3)))

##fileformat=VCFv4.2
##fileDate=20180201
##source=PLINKv1.90
##contig=<ID=1,length=282745832>
##contig=<ID=2,length=266367381>
##contig=<ID=3,length=177678048>
##contig=<ID=4,length=184213463>
##contig=<ID=5,length=173704786>
##contig=<ID=6,length=147965078>
##contig=<ID=7,length=145599967>
##contig=<ID=8,length=133288266>
##contig=<ID=9,length=122022420>
##contig=<ID=10,length=112580048>
##contig=<ID=11,length=90453650>
##contig=<ID=12,length=52683120>
##contig=<ID=13,length=114010850>
##contig=<ID=14,length=115436306>
##contig=<ID=15,length=111173208>
##contig=<ID=16,length=90610649>
##contig=<ID=17,length=90840848>
##contig=<ID=18,length=87963297>
##contig=<ID=19,length=62264106>
##contig=<ID=20,length=56089150>
##INFO=<ID=PR,Number=0,Type=Flag,Description="Provisional reference allele, may not be based on real reference genome">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	1052	1053	1054	1055	1059	1060	1061	1062	1065	106

Q: What's the gzip module for? [gzip docs](https://docs.python.org/3/library/gzip.html)

- This module provides a simple interface to compress and decompress files like the GNU program gzip
- `gzip`: a module that provides the `GzipFile` class as well as the `open`, `compress`, and `decompress` convenience functions.

Q: The "*.gz" file extension?

- .gz files: compressed files created using the gzip compression utility, which was created to replace and improve on compress in UNIX. This utility is commonly used on UNIX-like systems.
- gzip file compression is often used to compress some elements of web pages to speed up page loading. 

Q: Why is the tool called `gzip`? 

- A .gz file is an archive file compressed by the standard GNU zip (gzip) compression algorithm.

Q: Why use the `GzipFile` class? [class docs](https://docs.python.org/3/library/gzip.html#gzip.GzipFile)

- It simulates most of the methods of a "file object"

Q: "file object"? [Python docs. file object.](https://docs.python.org/3/glossary.html#term-file-object)

file object: 
- An object exposing a file-oriented API (with methods such as `read()` or `write()`) to an underlying resource
- Tutorial [file objects](https://youtu.be/Uh2ebFW8OYM)
- Tutorial [OS Module](https://www.youtube.com/watch?v=tJxcKyFMTGo)

Q: `gzip.open` method?



In [4]:
vcf_reader.metadata

OrderedDict([('fileformat', 'VCFv4.2'),
             ('fileDate', '20180201'),
             ('source', ['PLINKv1.90'])])

Q: How does the `OrderDict` type differ from the regular dictionary? [OrderedDict docs](https://docs.python.org/3.4/library/collections.html?highlight=ordereddict)
- It retains the order in which the entries were added.

[Python Dictionary Methods](https://www.w3schools.com/python/python_ref_dictionary.asp)

In [5]:
vcf_reader.metadata.items()

odict_items([('fileformat', 'VCFv4.2'), ('fileDate', '20180201'), ('source', ['PLINKv1.90'])])

In [6]:
for pair in vcf_reader.metadata.items():
    print(pair)

('fileformat', 'VCFv4.2')
('fileDate', '20180201')
('source', ['PLINKv1.90'])


In [7]:
vcf_reader.infos

OrderedDict([('PR',
              Info(id='PR', num=0, type='Flag', desc='Provisional reference allele, may not be based on real reference genome', source=None, version=None))])

In [8]:
vcf_reader.infos.items()

odict_items([('PR', Info(id='PR', num=0, type='Flag', desc='Provisional reference allele, may not be based on real reference genome', source=None, version=None))])

In [9]:
# name: name of an info object
# info: a vcf.Reader info object
for name, info in vcf_reader.infos.items():
    print(f"{name} ({info.type}): {info.desc}")

PR (Flag): Provisional reference allele, may not be based on real reference genome


`vcf_reader.infos` | info object:
- `info.type`: type of the info object
- `info.desc`: desription of the info object

## `vcf.model._Record` [[docs]][record docs]

**class `vcf.model._Record`**: A set of calls at a site. The standard VCF fields CHROM, POS, IS, REF, ALT, INFO, QUAL, FILTER, and FORMAT are available as properties (details on [Wikipedia][vcf_wikipedia]). 

The list of calls is in the `samples` attribute. 

[record docs]: https://pyvcf.readthedocs.io/en/latest/API.html#vcf-model-record
[vcf_wikipedia]: https://en.wikipedia.org/wiki/Variant_Call_Format

`vcf_reader` is an iterable object.

This means `it = iter(vcf_reader)` would be redundant and we can already use `next()`. 

In [10]:
def printRecordAttributes(vcf_reader, record=None, start=0):
    
    if record == None:
        for snp_idx, record in enumerate(vcf_reader, start=0):
            break
        
    print(f"Chromsome: {record.CHROM}")
    print(f"position: {record.POS}")
    print(f"alternative alleles: {record.ALT}")
    print(f"reference base: {record.REF}")
    print(f"Variation info: {record.INFO}")
    print(f"Identifier of variation: {record.ID}")

printRecordAttributes(vcf_reader)

Chromsome: 1
position: 175954
alternative alleles: [C]
reference base: G
Variation info: {'PR': True}
Identifier of variation: chr1.175954


In [11]:
def appendRecordData(record_df, record):
    """
    Args:
        record_df (pd.DataFrame): 
        record (vcf.model._Record):
    
    Returns:
        (pd.DataFrame): record_df with an additional row of record (SNP) data.
    """
    
    # Alternate allele bases
    if len(record.ALT) == 0:
        alt0, alt1 = np.nan, np.nan
    elif len(record.ALT) == 1:
        alt0, alt1 = record.ALT[0], np.nan

    varIdentifier = pd.Series(record.ID, name="varIdentifier")
    
    df = pd.DataFrame(
        data = {"refBase": record.REF, "altAllele0": alt0,
                "altAllele1": alt1},
        index = varIdentifier)
    
    record_df = record_df.append(df, ignore_index=False)
    
    return record_df

def appendCallData(call_df, record):
    """
    Args:
        call_df (pd.DataFrame):
        record (vcf.model._Record): 
        
    Returns:
        (pd.DataFrame): call_df with additional columns of (SNP) data
    """

    varIdentifier = pd.Series(record.ID, name="SNP"+"varIdentifier")
    sample_names = np.array([sample.sample for sample in record.samples])
    gt_types = np.array([sample.gt_type for sample in record.samples]).reshape(1,-1)
    
    df = pd.DataFrame(
        data = gt_types,
        columns = sample_names,
        index = varIdentifier)
    
    call_df = call_df.append(df, ignore_index=False)
    
    return call_df

In [12]:
def testAppendMethods():
    """Loops through the vcf file and converts the raw text data into 
    pd.DataFrame objects. 
    """
    vcf_reader = vcf.Reader(filename=vcf_file_path, compressed=True)
    
    # initialize DataFrames 
    recordAttributes = pd.DataFrame()
    call_df = pd.DataFrame()

    for snp_idx, record in enumerate(vcf_reader, start=0):
        recordAttributes = appendRecordData(recordAttributes, record=record)
        call_df = appendCallData(call_df, record=record) 

        # Stop criterion
        if snp_idx == 5:
            break

    print(recordAttributes.head(), '\n\n\n')
    print(call_df.head())

testAppendMethods()

              refBase altAllele0  altAllele1
varIdentifier                               
chr1.175954         G          C         NaN
chr1.175960         C          T         NaN
chr1.175970         T          A         NaN
chr1.177010         G          A         NaN
chr1.669529         T          C         NaN 



                  1052  1053  1054  1055  1059  1060  1061  1062  1065  1067  \
SNPvarIdentifier                                                               
chr1.175954          0     0     0     0     0     0     0     0     0     0   
chr1.175960          0     0     0     0     0     0     0     0     0     0   
chr1.175970          0     0     0     0     0     0     0     0     0     0   
chr1.177010          0     0     0     0     0     0     0     0     0     0   
chr1.669529          0     0     0     0     0     0     0     0     0     0   

                  ...  2028  1482  2442  3854  2057  2494  4022  4182  4659  \
SNPvarIdentifier  ...                    

## `vcf.model_Call` [[docs]][call docs]

**class `vcf.model._Call`**: A genotype call, a cell entry in a VCF file.

[call docs]: https://pyvcf.readthedocs.io/en/latest/API.html#vcf-model-call

### What exactly is genotype? And why is there a heterozygous property in this class?

A **gene** is a section of DNA that encodes a trait. The precise arrangement of **nucleotides** in a gene can differ between copies of the same gene. Therefore, a gene can exist in different forms across organisms. These different forms are known as **alleles**. The exact fixed position on the chromosome that contains a particular gene is known as a **locus**.

A **diploid** organism either inherits two copies of the same allele or one copy of two different alleles from their parents. If an individual inherits two identical alleles, their **genotype** is said to be **homozygous** at that locus. However, if they possess two different alleles, their genotype is classed as **heterozygous** for that locus.

Alleles of the same gene are either autosomal dominant or recessive. An **autosomal dominant allele** will always be preferentially expressed over a recessive allele.

The subsequent combination of alleles that an individual possesses for a specific gene is their **genotype**.  

Nucleotides are each composed of a phosphate group, sugar and a base.

[source: technologynetworks.com/genomics](https://www.technologynetworks.com/genomics/articles/genotype-vs-phenotype-examples-and-definitions-318446)

### Genotype calling, genotyping, and SNP calling?

What's the difference in these terms? What is "calling"?

**Source**: [[Kevin Blighe]](https://www.biostars.org/p/277927/)

#### Genotyping



#### Genotype & SNP calling





Genotype calling is the process of determining the genotype for each individual and is typically only done for positions in which a SNP or a 'variant' has already been called. We use the word 'calling' here to signify the estimation of one unique SNP or genotype.

**Source**: Nielsen, R., Paul, J. S., Albrechtsen, A., & Song, Y. S. (2011). Genotype and SNP calling from next-generation sequencing data. *Nature reviews. Genetics, 12(6)*, 443–451. [ncbi.nlm.nih.gov](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3593722/#:~:text=Genotype%20calling%20is%20the%20process,one%20unique%20SNP%20or%20genotype).

In [13]:
record = next(vcf_reader)
for idx, sample in enumerate(record.samples):
    print(sample.gt_bases, sample.gt_type)
    printRecordAttributes(vcf_reader=vcf_reader, record=record)
    print("\n")
    char = iter(sample.gt_bases)
    print(next(char))
    print(next(char))
    print(next(char))
    
    if idx == 1:
        break

C/C 0
Chromsome: 1
position: 175960
alternative alleles: [T]
reference base: C
Variation info: {'PR': True}
Identifier of variation: chr1.175960


C
/
C
C/C 0
Chromsome: 1
position: 175960
alternative alleles: [T]
reference base: C
Variation info: {'PR': True}
Identifier of variation: chr1.175960


C
/
C


We searched for a record with each `gt_type` to figure out what the variable meant. It's 0 if both alleles are the reference, 1 if one of them is, and 2 if both alleles are alternative alleles. 

```python
>>> df = pd.DataFrame(
        data = [[sample.sample, sample.gt_bases, 
            sample.gt_type, sample.is_het, 
            sample.phased] for sample in record.samples],
        columns = ["sample_name", "gt_bases", 
                   "gt_type", "is_het", "phased"])

>>> df['gt_bases'].value_counts()
A/A    1698
A/C      81
C/C       1
Name: gt_bases, dtype: int64
        
>>> df.groupby("gt_bases")['gt_type'].value_counts()
gt_bases  gt_type
A/A       0          1698
A/C       1            81
C/C       2             1
Name: gt_type, dtype: int64
        
>>> df['gt_type'].value_counts()
0    1698
1      81
2       1
Name: gt_type, dtype: int64
```

In [14]:
# vcf_reader.samples (list): the genotype calls

type(vcf_reader.samples), len(vcf_reader.samples), vcf_reader.samples[:3]

(list, 1780, ['1052', '1053', '1054'])

In [15]:
np.array(record.samples)

array([Call(sample=1052, CallData(GT=0/0)),
       Call(sample=1053, CallData(GT=0/0)),
       Call(sample=1054, CallData(GT=0/0)), ...,
       Call(sample=4182, CallData(GT=0/0)),
       Call(sample=4659, CallData(GT=0/0)),
       Call(sample=920, CallData(GT=0/0))], dtype=object)

In [16]:
print(f"Variant type: {record.var_type}\n\
      \t{record.is_snp}, {record.is_indel}\n\
      \t{record.alleles}")

Variant type: snp
      	True, False
      	['C', T]


---

## The Script 

In [17]:
import vcf

import os
import gzip
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use("ggplot")

def read_chars_gz(n):
    """ Read the first n characters of the .gz vcf file.
    Args:
        n (int)
    """
    with gzip.open(vcf_file_path, 'rt') as f:
        print(f.read(n))

def appendRecordData(record_df, record):
    """
    Args:
        record_df (pd.DataFrame): 
        record (vcf.model._Record):
    
    Returns:
        (pd.DataFrame): record_df with an additional row of record (SNP) data.
    """
    
    # Alternate allele bases
    if len(record.ALT) == 0:
        alt0, alt1 = np.nan, np.nan
    elif len(record.ALT) == 1:
        alt0, alt1 = record.ALT[0], np.nan

    varIdentifier = pd.Series(record.ID, name="varIdentifier")
    
    df = pd.DataFrame(
        data = {"refBase": record.REF, "altAllele0": alt0,
                "altAllele1": alt1},
        index = varIdentifier)
    
    record_df = record_df.append(df, ignore_index=False)
    
    return record_df

def appendCallData(call_df, record):
    """
    Args:
        call_df (pd.DataFrame):
        record (vcf.model._Record): 
        
    Returns:
        (pd.DataFrame): call_df with additional columns of (SNP) data
    """

    varIdentifier = pd.Series(record.ID, name="SNP"+"varIdentifier")
    sample_names = np.array([sample.sample for sample in record.samples])
    gt_types = np.array([sample.gt_type for sample in record.samples]).reshape(1,-1)
    
    df = pd.DataFrame(
        data = gt_types,
        columns = sample_names,
        index = varIdentifier)
    
    call_df = call_df.append(df, ignore_index=False)
    
    return call_df

def vcftoDataFrame(vcf_reader, verbose=False, test=False, save=False):
    """Loops through the vcf file and converts the raw text data into 
    pd.DataFrame objects. 
    """
    # time the operation
    start_time = time.time()
    
    # initialize DataFrames 
    recordAttributes = pd.DataFrame()
    call_df = pd.DataFrame()

    for snp_idx, record in enumerate(vcf_reader, start=0):
        recordAttributes = appendRecordData(recordAttributes, record=record)
        call_df = appendCallData(call_df, record=record) 
        
        if verbose == True:
            if (snp_idx % 1000) == 0:
                current_time = time.time() - start_time
                minutes = int(current_time / 60)
                seconds = current_time % 60
                print(f"SNPs looped: {snp_idx}. Time: {minutes} min, {seconds:.2f} s.")
            if (snp_idx % 10000) == 0:
                print("---------------------------")

        if test == True:
            # Stop criterion
            if snp_idx == 5:
                print(recordAttributes.head(), '\n\n\n')
                print(call_df.head())
                break
        
    if save == True:
        recordAttributes.to_csv("recordAttributes.csv")
        call_df.to_csv("gtTypes.csv")

vcftoDataFrame(vcf_reader, test=True, verbose=True, save=False)

SNPs looped: 0. Time: 0 min, 0.02 s.
---------------------------
              refBase altAllele0  altAllele1
varIdentifier                               
chr1.175970         T          A         NaN
chr1.177010         G          A         NaN
chr1.669529         T          C         NaN
chr1.669535         C          T         NaN
chr1.669543         A          C         NaN 



                  1052  1053  1054  1055  1059  1060  1061  1062  1065  1067  \
SNPvarIdentifier                                                               
chr1.175970          0     0     0     0     0     0     0     0     0     0   
chr1.177010          0     0     0     0     0     0     0     0     0     0   
chr1.669529          0     0     0     0     0     0     0     0     0     0   
chr1.669535          0     0     0     0     0     0     0     0     0     0   
chr1.669543          0     0     0     0     0     0     0     0     0     0   

                  ...  2028  1482  2442  3854  2057  24

In [18]:
def main():
    
    system_path = r"C:\Users\uniqu\Adaptation\github repos" \
              + "\Bioinformatics-Neural Networks for Genomic Risk"
    system_path = system_path + r"\DawleyRats"
    vcf_file_options = {
        "CharlesRiver": r"allChr.allSamps.90DR2.maf01.hweE7.noIBD.CharlesRiverOnly.vcf.gz",
        "Harlan": r"allChr.allSamps.90DR2.maf01.hweE7.noIBD.HarlanOnly.vcf.gz"}
    vcf_file_name = vcf_file_options["CharlesRiver"] # "...vcf.gz"
    vcf_file_path = os.path.join(system_path, vcf_file_name)
    vcf_reader = vcf.Reader(filename=vcf_file_path, compressed=True)

    try:
        print("VCF data loaded successfully...\n")
        print(f"Metadata:\n{vcf_reader.metadata}")
    except: 
        print("Failed to load VCF data.")    
        
    vcftoDataFrame(vcf_reader, test=False, verbose=True, save=True)    
    
if __name__ == '__main__':
    main()

VCF data loaded successfully...

Metadata:
OrderedDict([('fileformat', 'VCFv4.2'), ('fileDate', '20180201'), ('source', ['PLINKv1.90'])])
SNPs looped: 0. Time: 0 min, 0.03 s.
---------------------------
SNPs looped: 1000. Time: 0 min, 20.18 s.
SNPs looped: 2000. Time: 0 min, 42.12 s.
SNPs looped: 3000. Time: 1 min, 7.43 s.
SNPs looped: 4000. Time: 1 min, 36.73 s.
SNPs looped: 5000. Time: 2 min, 9.82 s.
SNPs looped: 6000. Time: 2 min, 51.89 s.
SNPs looped: 7000. Time: 3 min, 28.39 s.
SNPs looped: 8000. Time: 4 min, 11.51 s.
SNPs looped: 9000. Time: 4 min, 53.36 s.
SNPs looped: 10000. Time: 5 min, 39.17 s.
---------------------------
SNPs looped: 11000. Time: 6 min, 27.36 s.
SNPs looped: 12000. Time: 7 min, 18.38 s.
SNPs looped: 13000. Time: 8 min, 20.98 s.
SNPs looped: 14000. Time: 9 min, 16.16 s.
SNPs looped: 15000. Time: 10 min, 14.42 s.
SNPs looped: 16000. Time: 11 min, 18.50 s.
SNPs looped: 17000. Time: 12 min, 22.84 s.
SNPs looped: 18000. Time: 13 min, 28.96 s.
SNPs looped: 19000. 